In [26]:
import pyscf.tools
import glob
from pyscf import scf, gto, dft, mp, ao2mo, cc, mp, ci, fci
import json
from pyscf import gto,scf,mcscf, fci,lo
from pyscf.scf import ROHF, UHF, ROKS
import numpy as np
import pandas as pd
from pyscf.tools import fcidump
from pyscf.lib import chkfile

In [52]:
el = "CrO"
basis = "vqz"
this_chkfile = f'matts_chk_files/{el}_{basis}.chkfile'
mol = chkfile.load_mol(this_chkfile)
mf = scf.ROHF(mol)
scf_result_dic = chkfile.load(this_chkfile, 'scf')
mf.__dict__.update(scf_result_dic)

h1 = mf.get_hcore()
h2 = ao2mo.kernel(mol, mf.mo_coeff)

print(np.shape(h2))

(23005, 23005)


In [56]:
# Define the Pauli operations
I = 0
X = 1
Y = 2
Z = 3

@njit
def jordan_wigner_transform(hpq, eri_2idx, n_qubits):
    terms = []
    coeffs = []

    # Add one-body terms
    for p in range(n_qubits):
        for q in range(n_qubits):
            coeff = hpq[p, q]
            term = np.full(n_qubits, I)
            term[:p] = Z
            term[p] = X
            term[p+1:q] = Z
            term[q] = Y
            terms.append(term)
            coeffs.append(coeff)

    # Add two-body terms
    for pq in range(n_qubits**2):
        for rs in range(n_qubits**2):
            coeff = eri_2idx[pq, rs]

            p = pq // n_qubits
            q = pq % n_qubits
            r = rs // n_qubits
            s = rs % n_qubits

            term = np.full(n_qubits, I)
            term[:p] = Z
            term[p] = X
            term[p+1:q] = Z
            term[q] = X
            term[q+1:r] = Z
            term[r] = Y
            term[r+1:s] = Z
            term[s] = Y
            terms.append(term)
            coeffs.append(coeff)

    return terms, coeffs


In [ ]:
# two_to_four_index(v, no)

In [25]:
matts_Hs = glob.glob('matts_chk_files/*')
for chkfile_path in matts_Hs:
    
    

matts_chk_files/O0_v5z.chkfile
matts_chk_files/O1_v5z.chkfile
matts_chk_files/Sc0_v5z.chkfile
matts_chk_files/Sc1_v5z.chkfile
matts_chk_files/Ti0_v5z.chkfile
matts_chk_files/Ti1_v5z.chkfile
matts_chk_files/V0_v5z.chkfile
matts_chk_files/V1_v5z.chkfile
matts_chk_files/Cr0_v5z.chkfile
matts_chk_files/Cr1_v5z.chkfile
matts_chk_files/Mn0_v5z.chkfile
matts_chk_files/Mn1_v5z.chkfile
matts_chk_files/Fe0_v5z.chkfile
matts_chk_files/VO_v5z.chkfile
matts_chk_files/CrO_v5z.chkfile
matts_chk_files/MnO_v5z.chkfile
matts_chk_files/Fe1_v5z.chkfile
matts_chk_files/Cu0_v5z.chkfile
matts_chk_files/Cu1_v5z.chkfile
matts_chk_files/ScO_vdz.chkfile
matts_chk_files/TiO_vdz.chkfile
matts_chk_files/VO_vdz.chkfile
matts_chk_files/CrO_vdz.chkfile
matts_chk_files/MnO_vdz.chkfile
matts_chk_files/FeO_vdz.chkfile
matts_chk_files/CuO_vdz.chkfile
matts_chk_files/ScO_vtz.chkfile
matts_chk_files/TiO_vtz.chkfile
matts_chk_files/VO_vtz.chkfile
matts_chk_files/CrO_vtz.chkfile
matts_chk_files/MnO_vtz.chkfile
matts_chk_files

In [19]:
Hs = glob.glob('zapata_hams/*')

hpq, hpqrs = [], []
for local_filepath in Hs:
     # Read the fcidump file and print the ERI tensor.
    fci = pyscf.tools.fcidump.read(local_filepath)
    
    # Reshape the ERI into a 4-index tensor.
    eri = pyscf.ao2mo.restore("s1", fci["H2"], fci["H1"].shape[0])
    
    hpq.append(fci['H1'])
    hpqrs.append(eri)

Parsing zapata_hams/fcidump.28_2_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}
Parsing zapata_hams/fcidump.24_ts_1over2_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}
Parsing zapata_hams/fcidump.27_pc_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}
Parsing zapata_hams/fcidump.0_ru_macho_{'Ru': 'cc-pVTZ-PP', 'default': '6-311++G**'}_0.1
Parsing zapata_hams/fcidump.6_ts_ru_macho_melact_{'Ru': 'cc-pVTZ-PP', 'default': '6-311++G**'}
Parsing zapata_hams/fcidump.0_ru_macho_{'Ru': 'cc-pVTZ-PP', 'default': '6-311++G**'}_0.15
Parsing zapata_hams/fcidump.31_4a_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}
Parsing zapata_hams/fcidump.14_1_lut_prod_{'Mo': 'def2-SVP', 'default': '6-311+G(d,p)'}
Parsing zapata_hams/fcidump.19_I_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}
Parsing zapata_hams/fcidump.21_rc_{'Mo': 'def2-SVP', 'I': 

In [17]:
from numba import njit

In [ ]:
import numpy as np

# Define the Pauli operations
I = 0
X = 1
Y = 2
Z = 3

@njit
def jordan_wigner_transform(hpq, eri):
    n_qubits = np.shape(hpq)[0]
    terms = []
    coeffs = []

    # Add one-body terms
    for p in range(n_qubits):
        for q in range(n_qubits):
            coeff = hpq[p, q]
            term = np.full(n_qubits, I)
            term[:p] = Z
            term[p] = X
            term[p+1:q] = Z
            term[q] = Y
            terms.append(term)
            coeffs.append(coeff)

    # Add two-body terms
    for p in range(n_qubits):
        for q in range(n_qubits):
            for r in range(n_qubits):
                for s in range(n_qubits):
                    coeff = eri[p, q, r, s]
                    term = np.full(n_qubits, I)
                    term[:p] = Z
                    term[p] = X
                    term[p+1:q] = Z
                    term[q] = X
                    term[q+1:r] = Z
                    term[r] = Y
                    term[r+1:s] = Z
                    term[s] = Y
                    terms.append(term)
                    coeffs.append(coeff)

    return terms, coeffs


In [ ]:
def restrict_active_space(ne, no, hpq_full, eri_full):
    # Create an array of the indices of the active orbitals
    active_indices = np.arange(no)

    # Extract the sub-arrays corresponding to the active space
    hpq_active = hpq_full[np.ix_(active_indices, active_indices)]
    eri_active = eri_full[np.ix_(active_indices, active_indices, active_indices, active_indices)]

    return hpq_active, eri_active

In [22]:
for h in hpq:
    print(np.shape(h))

(21, 21)
(17, 17)
(37, 37)
(4, 4)
(36, 36)
(4, 4)
(29, 29)
(55, 55)
(60, 60)
(38, 38)
(40, 40)
(38, 38)
(10, 10)
(13, 13)
(46, 46)
(44, 44)
(44, 44)
(33, 33)
(44, 44)
(30, 30)
(27, 27)
(60, 60)
(4, 4)
(7, 7)
(36, 36)
(12, 12)
(15, 15)
(21, 21)
(38, 38)
(60, 60)
(46, 46)
(21, 21)
(27, 27)
(25, 25)
(4, 4)
(57, 57)
(14, 14)
(56, 56)


In [23]:
for i in range(len(hpq)):
    print('On ', i)
    jordan_wigner_transform(hpq[i], hpqrs[i])

On  0
On  1
On  2
On  3
On  4
On  5
On  6
On  7
On  8
On  9
On  10
On  11
On  12
On  13
On  14
On  15
On  16
On  17
On  18
On  19
On  20
On  21
On  22
On  23
On  24
On  25
On  26
On  27
On  28
On  29
On  30
On  31
On  32
On  33
On  34
On  35
On  36
On  37
